In [1]:
pip install pulp


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [4]:
segment_plan=pd.read_csv(" Campaign_Segment_Performance.csv")

In [6]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpBinary



# Calculate mailing cost
segment_plan["Mailing_Cost"] = segment_plan["total_customers"] * 0.25

# Setup optimization model
model = LpProblem("Campaign_Budget_Optimization", LpMaximize)


In [7]:
# Binary decision variables
decision_vars = {
    i: LpVariable(f"Select_{i}", cat=LpBinary)
    for i in segment_plan.index
}

In [8]:
# Objective: Maximize total actual revenue
model += lpSum(segment_plan.loc[i, "actual_revenue"] * decision_vars[i] for i in segment_plan.index)

In [10]:
# Constraint: total mailing cost ≤ 2 million
model += lpSum(segment_plan.loc[i, "Mailing_Cost"] * decision_vars[i] for i in segment_plan.index) <= 2_000_000
# Solve
model.solve()

1

In [11]:
# Store which segments were selected
segment_plan["Selected"] = [int(decision_vars[i].value()) for i in segment_plan.index]

# Save result
segment_plan.to_csv("Optimized_Campaign_Segments.csv", index=False)


In [12]:
#summary
print("Optimization Complete!")
print("Total Revenue:", segment_plan.loc[segment_plan["Selected"] == 1, "actual_revenue"].sum())
print("Total Cost:", segment_plan.loc[segment_plan["Selected"] == 1, "Mailing_Cost"].sum())
print("Total Expected Responses:", segment_plan.loc[segment_plan["Selected"] == 1, "expected_responses"].sum())


Optimization Complete!
Total Revenue: 23750468.2
Total Cost: 553.5
Total Expected Responses: 1212.2809125874126
